# Direct Radio Modem Test

In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [ ]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
sys.path.insert(0,"../tagfuse")
sys.path.insert(0,"../si446x/")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config, radio_poll
from tagfuse.radioutils import radio_get_rtctime, radio_set_rtctime

In [ ]:
from tagfuse.tagfuseargs import process_cmd_args, get_cmd_args
import tagfuse.tagfuseargs
tagfuse.tagfuseargs.global_args
tagfuse.tagfuseargs.global_args=None
#sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
sys.argv = ['tagfuse.py', '/home/pi/tag_store']
process_cmd_args()
get_cmd_args()
from tagfuse.TagFuseTree import TagFuseTagTree

In [ ]:
sys.path

In [ ]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

In [ ]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
#radio = radio_start()
from si446x import Si446xRadio
from si446x import get_config_wds, get_name_wds, wds_default_config
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
wds_default_config(0) # force alternate default config
radio.write_config()
radio.config_frr()

In [ ]:
radio_show_config(radio.dump_radio())

In [ ]:
from si446x import get_name_wds
get_name_wds()

### Set Defaults

In [ ]:
# default paramters
MAX_WAIT            = 1000
MAX_RECV            = 2
MAX_PAYLOAD         = 254
MAX_RETRIES         = 3
RADIO_POWER         = 20
SHORT_DELAY         = 1

## Send Packets

In [ ]:
STOP


In [ ]:
SEND_DELAY=.3
pwr = 120
msg = bytearray('0' + ('0123456789' * 20))
msg[0] = len(msg) - 1
for i in range(20000):
    msg[1] = i % 256
    radio_send_msg(radio, msg, pwr)
    sleep(SEND_DELAY)

## Listen for packets

In [ ]:
import datetime
fails = 0
last = 0
pause = 10
start = True
msg = bytearray()
for i in range(200000):
    msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 10)
    if msg:
        if (((msg[0] + 1) != len(msg)) or (msg[1] != last % 256)):
            if start:
                start = False
            else:
                fails += 1
                print('error', fails, rssi, msg[0], len(msg), msg[1], last, end='\r')
            last = msg[1] 
        last += 1
    else:
        fails += 1
        print('timeout', i, fails)
    if (i % pause) == 0:
        print('{}, i:{}, len:{}, rssi:{}, last:{}, fails:{}    '.format(datetime.datetime.now(), i, len(msg), rssi, last % 256, fails), end='\r')


In [ ]:
status

In [ ]:
i, fails

## Functions for Accessing Internal Radio State

In [ ]:
STOP

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from si446xdef import *


def si446x_device_group_fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_device_get_property(radio, gname, 0, g_s.sizeof())
    print(g_s, insert_space(p))
    #print(gname, len(p), hexlify(p))
    print(radio_display_structs[g_s](g_s, p))
    return None


def si446x_device_command_fetch_and_decode(cmd):
    cname = radio_status_cmd_ids.decoding[cmd]
    cfunc, cstr = radio_status_commands[radio_config_cmd_ids.build(cname)]
#    print(cname, cmd)
    if (cfunc):
        cmd = cfunc(cname)
        if (cmd):
#            print(cfunc, hexlify(cmd), cstr)
            radio.spi.command(cmd, cstr)
            rsp = radio.spi.response(cstr.sizeof(), cstr.name)
            if (rsp):
#                print(cstr, radio_display_structs[cstr])
                print(cstr, insert_space(rsp))
                print(radio_display_structs[cstr](cstr, rsp))
            else:
                print('no response')
        else:
            print('no command')
    else:
        print('no function')
    return None

def si446x_device_get_property(radio, g_n, start, limit):
    prop_x = 0
    prop_b = bytearray()
    while (prop_x < limit):
        chunk_size = limit - prop_x
        x = MAX_RADIO_RSP if (chunk_size >= MAX_RADIO_RSP) else chunk_size
        rsp = radio.get_property(g_n, prop_x, x)
        if (rsp):
            prop_b += bytearray(rsp[:x])
            prop_x += x
        else:
            return None
    return prop_b

## Interactive Group Properties

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
STOP

In [ ]:
import time
ix = 100
tt = time.time()
for i in range(ix): x = 0
d_tt = time.time() - tt
ct = time.clock()
for i in range(ix): y = 0
d_ct = time.clock() - ct
d_ct/num_iters, d_tt/num_iters